<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [2]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10173735791472240765, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11154422528
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5318758604266751051
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [2]:
!rm concrete_data_week4.zip concrete_data_week4.zip.1

!rm -r concrete_data_week4

!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

rm: cannot remove 'concrete_data_week4.zip': No such file or directory
rm: cannot remove 'concrete_data_week4.zip.1': No such file or directory
rm: cannot remove 'concrete_data_week4': No such file or directory
--2021-04-07 09:28:23--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  25.2MB/s    in 9.3s    

2021-04-07 09:28:33 (26.9 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [3]:
!unzip concrete_data_week4.zip > unzip_output.out

print('unzipped')

unzipped


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

  # I had some problems in the previous lab in order to train the model because the lab kept crashing therefore I am training both models here

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [6]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.applications import VGG16 as VVG16 #had a typo
from keras.applications import ResNet50

from keras.applications.vgg16 import preprocess_input as vvg16_preprocess_input
from keras.applications.resnet50 import preprocess_input as resnet50_preprocess_input

In [9]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

## training vvg16

In [7]:
vvg16_data_generator = ImageDataGenerator(
    preprocessing_function=vvg16_preprocess_input,
)

In [10]:
vvg16_train_generator = vvg16_data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [9]:
vvg16_validation_generator = vvg16_data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [10]:
vvg16_model = Sequential()

vvg16_model.add(VVG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

vvg16_model.add(Dense(num_classes, activation='softmax'))

vvg16_model.layers[0].trainable = False

vvg16_model.summary()

58892288/58889256 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [11]:
vvg16_model.layers

In [12]:
vvg16_model.layers[0].layers

In [13]:
vvg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
vvg16_steps_per_epoch_training = len(vvg16_train_generator)
vvg16_steps_per_epoch_validation = len(vvg16_validation_generator)
num_epochs = 2

In [15]:
vvg16_fit_history = vvg16_model.fit_generator(
    vvg16_train_generator,
    steps_per_epoch=vvg16_steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=vvg16_validation_generator,
    validation_steps=vvg16_steps_per_epoch_validation,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
301/301 [==============================] - 342s 958ms/step - loss: 0.4285 - accuracy: 0.8620 - val_loss: 0.0377 - val_accuracy: 0.9924
Epoch 2/2
301/301 [==============================] - 287s 954ms/step - loss: 0.0334 - accuracy: 0.9933 - val_loss: 0.0217 - val_accuracy: 0.9952


In [16]:
vvg16_model.save('classifier_vvg16_model.h5')

## training resnet 50

because the previous lab kept crushing, so I have not been able to train it

In [11]:
resnet50_data_generator = ImageDataGenerator(
    preprocessing_function=resnet50_preprocess_input,
)

In [18]:
resnet50_train_generator = resnet50_data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [19]:
resnet50_validation_generator = resnet50_data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [20]:
resnet50_model = Sequential()

resnet50_model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

resnet50_model.add(Dense(num_classes, activation='softmax'))

resnet50_model.layers[0].trainable = False

resnet50_model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [21]:
resnet50_model.layers

In [22]:
resnet50_model.layers[0].layers

In [23]:
resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
resnet50_steps_per_epoch_training = len(resnet50_train_generator)
resnet50_steps_per_epoch_validation = len(resnet50_validation_generator)
#num_epochs = 2

In [25]:
resnet50_fit_history = resnet50_model.fit_generator(
    resnet50_train_generator,
    steps_per_epoch=resnet50_steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=resnet50_validation_generator,
    validation_steps=resnet50_steps_per_epoch_validation,
    verbose=1
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
301/301 [==============================] - 258s 837ms/step - loss: 0.0574 - accuracy: 0.9811 - val_loss: 0.0057 - val_accuracy: 0.9988
Epoch 2/2
301/301 [==============================] - 250s 831ms/step - loss: 0.0050 - accuracy: 0.9988 - val_loss: 0.0047 - val_accuracy: 0.9985


In [26]:
vvg16_model.save('classifier_resnet50_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [12]:
resnet50_test_generator = resnet50_data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

vvg16_test_generator = vvg16_data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [28]:
resnet_result = resnet50_model.evaluate_generator(resnet50_test_generator)

print('resnet ', resnet_result)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


resnet  [0.002283327979966998, 1.0]


In [29]:
vvg16_result = vvg16_model.evaluate_generator(vvg16_test_generator)

print('vvg16 ', vvg16_result)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


vvg16  [0.019957879558205605, 0.9959999918937683]


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

# Had a small annoing problem with the notebook therefore I reload the models



In [4]:
import keras

In [14]:
resnet50_model = keras.models.load_model('classifier_resnet50_model.h5')

resnet_result = resnet50_model.predict_generator(resnet50_test_generator)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [15]:
def print_pos_neg(result):

  if result[0] > result[1]:

    print('Negative')

  else:

    print('Positive')

print('resnet:')

for i in range(5):

  print_pos_neg(resnet_result[i])

resnet:
Negative
Negative
Negative
Negative
Negative


In [16]:
vvg16_model = keras.models.load_model('classifier_vvg16_model.h5')

vvg16_result = vvg16_model.predict_generator(vvg16_test_generator)

print('vvg16:')

for i in range(5):

  print_pos_neg(vvg16_result[i])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


vvg16:
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).